In [ ]:
# Cellule 1: Installation des dépendances
!pip install git+https://github.com/2nzi/EVA2SPORT.git@dev-pipeline-eva2sport
!pip install git+https://github.com/facebookresearch/sam2.git
!pip install opencv-python torch

# Nettoyage des dossiers de test
!rm -rf /content/data/videos/outputs/*
!rm -rf /content/sample_data/*

print("✅ Installation terminée !")

In [ ]:
# Cellule 2: Téléchargement du modèle SAM2
!mkdir -p checkpoints
!wget -O checkpoints/sam2.1_hiera_large.pt https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt

# Vérification
!ls -lh checkpoints/sam2.1_hiera_large.pt
print("✅ Modèle SAM2 téléchargé !")

In [ ]:
# Cellule 3: Imports et configuration globale
from eva2sport import EVA2SportPipeline
from eva2sport.export.multi_event_manager import MultiEventManager
import os
from pathlib import Path

# Configuration globale - MODIFIEZ SELON VOS BESOINS
VIDEO_NAME = "SD_13_06_2025_cam1"  # ⚠️ Nom de base de votre vidéo
WORKING_DIR = "/content"

print(f"🎯 Configuration:")
print(f"   📹 Vidéo: {VIDEO_NAME}")
print(f"   📁 Répertoire: {WORKING_DIR}")

In [ ]:
# =============================================================================
# 🔍 SECTION 2: VÉRIFICATION DES PRÉREQUIS
# =============================================================================

In [ ]:
# Cellule 4: Vérification des fichiers uploadés
def check_required_files():
    """Vérifie que tous les fichiers nécessaires sont présents"""
    base_path = Path(WORKING_DIR)
    errors = []
    
    # Fichiers requis
    required_files = [
        f"data/videos/{VIDEO_NAME}.mp4",
        f"data/videos/{VIDEO_NAME}_calib.json",
        f"data/videos/{VIDEO_NAME}_objects.json",
    ]
    
    print("🔍 Vérification des fichiers requis...")
    for file_path in required_files:
        full_path = base_path / file_path
        if full_path.exists():
            size = full_path.stat().st_size / (1024*1024)  # MB
            print(f"   ✅ {file_path} ({size:.1f} MB)")
        else:
            print(f"   ❌ {file_path} - MANQUANT")
            errors.append(file_path)
    
    # Fichiers optionnels
    optional_files = [
        f"data/videos/Timeline_g_SD.csv",  # Pour traitement CSV
    ]
    
    print("\n📋 Fichiers optionnels:")
    for file_path in optional_files:
        full_path = base_path / file_path
        if full_path.exists():
            print(f"   ✅ {file_path}")
        else:
            print(f"   ⚠️  {file_path} - optionnel")
    
    if errors:
        print(f"\n❌ {len(errors)} fichier(s) manquant(s) - uploadez-les avant de continuer!")
        return False
    else:
        print(f"\n✅ Tous les fichiers requis sont présents!")
        return True

# Vérification
files_ok = check_required_files()

In [ ]:
# =============================================================================
# 🚀 SECTION 3: CAS D'USAGE 1 - PIPELINE SUR UN SEGMENT SPÉCIFIQUE
# =============================================================================

In [ ]:
# Cellule 5: Pipeline sur un segment d'événement spécifique
if not files_ok:
    print("❌ Fichiers manquants - ne peut pas continuer")
else:
    print("🎯 CAS D'USAGE 1: PIPELINE SUR UN SEGMENT SPÉCIFIQUE")
    print("=" * 60)
    
    # Configuration du segment
    EVENT_TIMESTAMP = 959  # secondes ⚠️ MODIFIEZ SELON VOTRE ÉVÉNEMENT
    OFFSET_BEFORE = 10.0   # secondes avant l'événement
    OFFSET_AFTER = 5.0     # secondes après l'événement
    
    try:
        # Créer la pipeline en mode segment
        print(f"🏗️ Création pipeline segment ({EVENT_TIMESTAMP}s ±{OFFSET_BEFORE}/{OFFSET_AFTER}s)...")
        pipeline = EVA2SportPipeline(
            VIDEO_NAME,
            working_dir=WORKING_DIR,
            event_timestamp_seconds=EVENT_TIMESTAMP,
            segment_offset_before_seconds=OFFSET_BEFORE,
            segment_offset_after_seconds=OFFSET_AFTER
        )
        
        # Exécution avec export vidéo
        print("🚀 Exécution pipeline avec export vidéo...")
        results_segment = pipeline.run_full_pipeline(
            force_extraction=True,
            export_video=True,
            video_params={
                'fps': 10,                    # FPS pour la visualisation
                'show_minimap': True,         # Inclure la minimap
                'cleanup_frames': False,      # Garder les frames pour debug
                'force_regenerate': True      # Forcer régénération
            }
        )
        
        print("✅ Pipeline segment terminée!")
        print(f"📊 Résultats: {len(results_segment.get('tracked_objects', {}))} objets trackés")
        
    except Exception as e:
        print(f"❌ Erreur pipeline segment: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# =============================================================================
# 🎬 SECTION 4: CAS D'USAGE 2 - PIPELINE COMPLÈTE SUR TOUTE LA VIDÉO
# =============================================================================

In [ ]:
# Cellule 6: Pipeline sur la vidéo complète (ATTENTION: peut être très long)
if not files_ok:
    print("❌ Fichiers manquants - ne peut pas continuer")
else:
    print("🎬 CAS D'USAGE 2: PIPELINE COMPLÈTE SUR TOUTE LA VIDÉO")
    print("=" * 60)
    print("⚠️  ATTENTION: Cela peut prendre très longtemps selon la durée de la vidéo!")
    
    # Demander confirmation (simulé ici)
    PROCESS_FULL_VIDEO = True  # ⚠️ Changez en True pour traiter toute la vidéo
    
    if not PROCESS_FULL_VIDEO:
        print("🛑 Traitement vidéo complète désactivé (PROCESS_FULL_VIDEO = False)")
    else:
        try:
            # Créer la pipeline SANS offsets = toute la vidéo
            print("🏗️ Création pipeline vidéo complète...")
            pipeline_full = EVA2SportPipeline(
                VIDEO_NAME,
                working_dir=WORKING_DIR
                # Pas d'offsets = toute la vidéo
            )
            
            print("🚀 Exécution pipeline complète...")
            print("   ⏳ Cela peut prendre 10-30 minutes selon la vidéo...")
            
            results_full = pipeline_full.run_full_pipeline(
                force_extraction=True,
                export_video=True,
                video_params={
                    'fps': 5,                     # FPS réduit pour vidéo longue
                    'show_minimap': True,
                    'cleanup_frames': True,       # Nettoyer pour économiser l'espace
                    'force_regenerate': False     # Ne pas forcer si déjà fait
                }
            )
            
            print("✅ Pipeline vidéo complète terminée!")
            print(f"📊 Résultats: {len(results_full.get('tracked_objects', {}))} objets trackés")
            
        except Exception as e:
            print(f"❌ Erreur pipeline complète: {e}")
            import traceback
            traceback.print_exc()

In [ ]:
# =============================================================================
# 📊 SECTION 5: CAS D'USAGE 3 - TRAITEMENT MULTI-ÉVÉNEMENTS DEPUIS CSV
# =============================================================================

In [ ]:
# Cellule 7: Traitement de plusieurs événements depuis un fichier CSV
if not files_ok:
    print("❌ Fichiers manquants - ne peut pas continuer")
else:
    print("📊 CAS D'USAGE 3: TRAITEMENT MULTI-ÉVÉNEMENTS DEPUIS CSV")
    print("=" * 60)
    
    CSV_FILE = "Timeline_g_SD.csv"  # ⚠️ Nom de votre fichier CSV
    csv_path = Path(WORKING_DIR) / "data" / "videos" / CSV_FILE
    
    if not csv_path.exists():
        print(f"⚠️  Fichier CSV non trouvé: {CSV_FILE}")
        print("   Vous pouvez continuer avec les autres cas d'usage")
    else:
        try:
            print(f"📋 Traitement des événements depuis: {CSV_FILE}")
            
            # Créer le gestionnaire multi-événements
            manager = MultiEventManager(VIDEO_NAME, working_dir=WORKING_DIR)
            
            # Configuration du traitement
            results_csv = manager.process_events_from_csv(
                csv_file=CSV_FILE,
                timestamp_column='Start time',       # ⚠️ Nom de votre colonne timestamp
                filter_column='Row',                 # ⚠️ Colonne de filtrage
                filter_value='PdB',                  # ⚠️ Valeur à filtrer
                segment_offset_before_seconds=10.0,
                segment_offset_after_seconds=5.0,
                video_params={
                    'fps': 8,
                    'show_minimap': True,
                    'cleanup_frames': True,
                    'force_regenerate': False
                }
            )
            
            print("✅ Traitement multi-événements terminé!")
            print(f"📊 {len(results_csv)} événements traités")
            
            # Afficher un résumé
            for event_name, result in results_csv.items():
                status = "✅" if result.get('success', False) else "❌"
                objects_count = len(result.get('tracked_objects', {}))
                print(f"   {status} {event_name}: {objects_count} objets")
                
        except Exception as e:
            print(f"❌ Erreur traitement CSV: {e}")
            import traceback
            traceback.print_exc()

In [ ]:
# Sauvegarde complète sur Google Drive
from google.colab import drive
import shutil
from datetime import datetime

# Monter Google Drive
drive.mount('/content/gdrive')

# Créer le dossier de backup avec timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_path = f"/content/gdrive/MyDrive/EVA2SPORT_backup_{timestamp}"

# Copier tout le dossier data
print("🚀 Sauvegarde en cours...")
shutil.copytree("/content/data", f"{backup_path}/data")

print(f"✅ Sauvegarde terminée dans: EVA2SPORT_backup_{timestamp}")